In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_dataset.csv')
data["date"] = pd.to_datetime(data["date"])
data

,date,time,target,temp,temp_pred,weather_pred,weather_fact
0,2023-04-01,0,479.282,5.7,5.0,"пасм, 58%","пасм, морось"
1,2023-04-01,1,445.182,5.7,5.0,"пасм, 58%","пасм, морось"
2,2023-04-01,2,424.225,5.7,5.0,"пасм, 58%","пасм, морось"
3,2023-04-01,3,413.866,5.0,4.0,"пасм, 71% дождь","пасм, дымка"
4,2023-04-01,4,408.146,5.0,4.0,"пасм, 71% дождь","пасм, дымка"
...,...,...,...,...,...,...,...
2923,2023-07-31,19,453.173,17.2,18.0,"пасм, 24% дождь",п/обл
2924,2023-07-31,20,446.287,17.2,18.0,"пасм, 24% дождь",п/обл
2925,2023-07-31,21,444.373,15.8,17.0,"пасм, 24% дождь",п/обл
2926,2023-07-31,22,449.078,15.8,17.0,"пасм, 24% дождь",п/обл


In [ ]:
features = data[["date", "time", "temp"]]
target = data["target"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
# import sklearn
#scaler = sklearn.preprocessing.RobustScaler()
X_train[['time', 'temp']] = scaler.fit_transform(X_train[['time', 'temp']])
X_test[['time', 'temp']] = scaler.transform(X_test[['time', 'temp']])

In [ ]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [ ]:
numeric_features = ["time", "temp"]
X_train = X_train[numeric_features]
X_test = X_test[numeric_features]

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Класс нейросети

In [ ]:
class MyModel(nn.Module):
    def __init__(self, input_dim):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
input_dim = X_train.shape[1]
model = MyModel(input_dim)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Эпоха [{epoch+1}/{num_epochs}], Потеря: {loss.item()}')


model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    print("Потеря на тестовых данных:", test_loss.item())


threshold = 1.0
predictions = model(X_test)
mse = nn.MSELoss()(predictions, y_test)
rmse = torch.sqrt(mse)
successful_predictions = (rmse < threshold).sum().item()
total_predictions = len(y_test)
success_rate = (successful_predictions / total_predictions) * 100
print(f"Процент успешных предсказаний (RMSE меньше {threshold}): {success_rate:.2f}%")

Эпоха [10/100], Потеря: 155794.296875
Эпоха [20/100], Потеря: 150596.75
Эпоха [30/100], Потеря: 144900.65625
Эпоха [40/100], Потеря: 138795.8125
Эпоха [50/100], Потеря: 132317.546875
Эпоха [60/100], Потеря: 125495.2578125
Эпоха [70/100], Потеря: 118360.5
Эпоха [80/100], Потеря: 110870.03125
Эпоха [90/100], Потеря: 103102.484375
Эпоха [100/100], Потеря: 95166.5
Потеря на тестовых данных: 91006.4921875
Процент успешных предсказаний (RMSE меньше 1.0): 0.00%
